In [2]:
import numpy as np 
import statsmodels.api as sm

In [32]:
num = 1000 
 
event_time = int(num / 2) 
 
R_market = np.random.normal(0, 1, num) + np.arange(num) / num 
 
R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == int(num / 2) + 1) * 2 
 
results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit() 
 
alpha, beta = results.params 
 
resid = R_target - results.predict(sm.add_constant(R_market)) 
 
print(resid[event_time + 1] / resid[:event_time].std(ddof = 2)) 

1.8596250498156004


In [40]:
# Initialize
np.random.seed(0)

# Constants
num = 1000
true_event_time = int(num / 2)

# Generate market returns with slight trend
R_market = np.random.normal(0, 1, num) + np.arange(num) / num

# Generate target returns with an abnormal spike at true_event_time + 1
R_target = (
    2
    + R_market
    + np.random.normal(0, 1, num)
    + (np.arange(num) == (true_event_time + 1)) * 2
)

# Perform placebo tests
placebo_t_stats = []

for placebo_event_time in range(100, num - 2):  # Leave room for a post-event residual
    np.random.seed(0)  # Fix the seed at each iteration

    # Use pre-placebo-event data to train the model
    results = sm.OLS(R_target[:placebo_event_time], sm.add_constant(R_market[:placebo_event_time])).fit()

    # Compute residuals
    resid = R_target - results.predict(sm.add_constant(R_market))

    # Get standardized residual at the (fictitious) event time + 1
    t_stat = resid[placebo_event_time + 1] / resid[:placebo_event_time].std(ddof=2)
    placebo_t_stats.append(t_stat)

# Compute fraction of placebo tests where |t| > 1.96
detected_fraction = np.mean(np.abs(placebo_t_stats) > 1.96)

detected_fraction

np.float64(0.045657015590200446)

In [41]:
# Set parameters
num = 1000
true_event_time = int(num / 2)
n_runs = 100  # Number of repeated experiments to average over
n_placebos = 20  # Placebo windows before and after the actual event

# Store the fraction of placebo t-values exceeding the actual event t-value for each run
fractions = []

for run in range(n_runs):
    np.random.seed(run)  # New dataset each time

    # Generate market and target returns
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num
    R_target = (
        2
        + R_market
        + np.random.normal(0, 1, num)
        + (np.arange(num) == (true_event_time + 1)) * 2
    )

    # Get actual event t-value
    results_actual = sm.OLS(R_target[:true_event_time], sm.add_constant(R_market[:true_event_time])).fit()
    resid_actual = R_target - results_actual.predict(sm.add_constant(R_market))
    actual_t = resid_actual[true_event_time + 1] / resid_actual[:true_event_time].std(ddof=2)

    # Store placebo t-values
    placebo_t_values = []
    placebo_times = list(range(true_event_time - n_placebos, true_event_time)) + \
                    list(range(true_event_time + 1, true_event_time + 1 + n_placebos))

    for placebo_time in placebo_times:
        # Train only on data before this placebo time
        results = sm.OLS(R_target[:placebo_time], sm.add_constant(R_market[:placebo_time])).fit()
        resid = R_target - results.predict(sm.add_constant(R_market))
        t_stat = resid[placebo_time + 1] / resid[:placebo_time].std(ddof=2)
        placebo_t_values.append(t_stat)

    # Compute the fraction of placebo t-values greater than actual
    fraction_higher = np.mean(np.abs(placebo_t_values) > np.abs(actual_t))
    fractions.append(fraction_higher)

# Compute the average fraction across runs
average_fraction = np.mean(fractions)

average_fraction

np.float64(0.13924999999999998)

In [47]:
def make_error(corr_const, num): 
 
  sigma = 5 * 1 / np.sqrt((1 - corr_const)**2 / (1 - corr_const**2)) 
 
  err = list() 
 
  prev = np.random.normal(0, sigma) 
 
  for n in range(num): 
 
    prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, sigma) 
 
    err.append(prev) 
 
  return np.array(err) 

In [48]:
# Parameters
num = 1000
true_event_time = int(num / 2)
n_runs = 100  # Number of repeated experiments
n_placebos = 20
corr_const = 0.9

# Store results
fractions = []

for run in range(n_runs):
    np.random.seed(run)  # Different dataset each time

    # Generate market returns
    R_market = np.random.normal(0, 1, num) + np.arange(num) / num

    # Generate autocorrelated errors for R_target
    error = make_error(corr_const, num)

    # Inject true event at true_event_time + 1
    R_target = 2 + R_market + error + (np.arange(num) == (true_event_time + 1)) * 2

    # Actual event t-statistic
    results_actual = sm.OLS(R_target[:true_event_time], sm.add_constant(R_market[:true_event_time])).fit()
    resid_actual = R_target - results_actual.predict(sm.add_constant(R_market))
    actual_t = resid_actual[true_event_time + 1] / resid_actual[:true_event_time].std(ddof=2)

    # Placebo test
    placebo_t_values = []
    placebo_times = list(range(true_event_time - n_placebos, true_event_time)) + \
                    list(range(true_event_time + 1, true_event_time + 1 + n_placebos))

    for placebo_time in placebo_times:
        results = sm.OLS(R_target[:placebo_time], sm.add_constant(R_market[:placebo_time])).fit()
        resid = R_target - results.predict(sm.add_constant(R_market))
        t_stat = resid[placebo_time + 1] / resid[:placebo_time].std(ddof=2)
        placebo_t_values.append(t_stat)

    # Fraction of placebo t-values greater than actual
    fraction_higher = np.mean(np.abs(placebo_t_values) > np.abs(actual_t))
    fractions.append(fraction_higher)

# Average fraction across runs
average_fraction_corr = np.mean(fractions)

average_fraction_corr

np.float64(0.50475)